In [ ]:
!pip install newsdataapi newspaper3k lxml_html_clean

In [ ]:
from newsdataapi import NewsDataApiClient
from newspaper import Article
import json
from datetime import datetime 

# 1️⃣ ตั้งค่า API
api = NewsDataApiClient(apikey="pub_70b63fee186742658db1dfb76d4a52c7")

# 2️⃣ กำหนดคีย์เวิร์ดที่สนใจ
keywords = ["Nvidia", "AMD", "Intel"]

# 3️⃣ เตรียม dictionary ตาม format ที่ต้องการ
all_articles = {key: [] for key in keywords}

for keyword in keywords:
    print(f"Fetching news for {keyword}...")
    response = api.news_api(
        q=keyword,
        category="business,lifestyle,technology,top,science",
        language="en",
        country="us"
    )

    for news in response.get('results', []):
        url = news.get('link')
        source_name = news.get('source_id', '')  # ใช้ source จาก API
        try:
            article = Article(url)
            article.download()
            article.parse()

            # แปลง publish_date ให้เป็น date (YYYY-MM-DD)
            if article.publish_date:
                timestamp = article.publish_date.date().isoformat()
            else:
                timestamp = None  # หรือ "" ถ้าต้องการ string ว่าง

            news_data = {
                "headline": article.title,
                "content": article.text,
                "source": source_name,
                "url": url,
                "timestamp": timestamp
            }

            all_articles[keyword].append(news_data)
            print(f"Added: {article.title}")

        except Exception as e:
            print(f"Error downloading {url}: {e}")

# 4️⃣ สร้างชื่อไฟล์จากวันปัจจุบัน
today_str = datetime.now().strftime("%d%m%Y")  # เช่น 19092025
filename = f"{today_str}.json"

# 5️⃣ บันทึกเป็นไฟล์ JSON
with open(filename, "w", encoding="utf-8") as f:
    json.dump(all_articles, f, ensure_ascii=False, indent=2)

print(f"Saved to {filename}")
